# Homework 2 Solutions: Integrable Singularities and Monte Carlo Integration
### Gabe Schumm
### Due October 6th, 2021

# 1) Integrable Singularities

In [ ]:
using Plots, Statistics, DelimitedFiles, LsqFit, LaTeXStrings, Printf
#Plots for plotting
#Statistics for mean
#DelimitedFiles to write data (not used here)
#LsqFit to fit to do straight line fit
#LaTeXStrings for nice plot font
#Printf for string formating

I will first define a module that will contain the Integrable Singularities structure `IntSing`. The reason I do this is that if we want to edit our struct in any way (add a new field for example) we can do so in the structure defined within the module with not issues. If the structure was defined globally, it will be uneditable and you would have to restart your Kernel in order to adjust it.

I add `mutable` in front of `struct` so that the values of the fields for a certain instance can be changed after its initially defined. For this function. This isn't necessary but in other cases, this will be helpful.

In [ ]:
module IntSings

mutable struct IntSing #define the structure IntSing that will contain the specifications of the integration
    ϵ::Float64
    α::Float64
    
    x0::Float64
    xf::Float64

    N0::Int64
    N::Int64
    n::Int64

    h::Float64
    
    ifxn::Any #function we will use for integration (either first or second order)
end


#define first order integration function within module
function int1(self)
    function f(x) return 1/(self.ϵ + x)^self.α end
        
    I = 1.5*(f(self.x0+self.h) + f(self.xf-self.h)) # add end points
    
    for i=2:self.N-2 # n must be even
        I += f(self.x0+ (self.h* i))
    end
    return I * self.h
end
   
#define second order integration function within module
function int2(self)
    function f(x) return 1/(self.ϵ + x)^self.α end
        
    I = ((23/12)*(f(self.x0+self.h) + f(self.xf-self.h)) +
        (7/12)*(f(self.x0+(2*self.h)) + f(self.xf-(2*self.h))))# add end points
    
    for i=3:self.N-3 # n must be even
        I += f(self.x0+ (self.h* i))
    end
    return I * self.h
end


#function with same name as structure, used to initialize structure instance and set constants
function IntSing(ϵ, α, n, ifxn_id)
    
    #these are the same for all runs
    x0 = 0
    xf = 1
    N0 = 10
    
    N = N0 * 2^n
    
    #if user input ifxn_id=1, use first order, if 2 use second order
    if ifxn_id == 1
        ifxn = int1
    elseif ifxn_id == 2
        ifxn = int2
    else
        error("Invalid ifxn_id.")
    end

            
    h = (xf - x0)/N
         
    #return instance of IntSing with user specified and default variables
    return IntSing(ϵ, α,
                x0, xf,
                N0, N, n,
                h,
                ifxn)
    

end

end

#define IntSing as IntSing from module IntSings in global scope
IntSing = IntSings.IntSing

## Part a

In [1]:
linear(x, p) = p[1] .+ x .* p[2] #function used in curve fit (y=mx+b)

linear (generic function with 1 method)

In [ ]:
hs = []
I1 = [] #first order
I2 = [] #second order

ϵ = 1
α = 0.5
for n=0:11
    intsing1 = IntSing(ϵ, α, n, 1) #define instance with first order integral funciton
    push!(I1, intsing1.ifxn(intsing1))
    
    intsing2 = IntSing(ϵ, α, n, 2) #define instance with second order integral funciton
    push!(I2, intsing2.ifxn(intsing2))
    
    push!(hs, intsing2.h)
end

I_exact = 2^(1.5) - 2 #exact formula for integral

#fit straight line to log-log data with arbitrary initial parameters (.5, .5)
fit_1 = curve_fit(linear, log.(hs), log.(abs.(I1 .- I_exact)), [.5,.5])                                      
fit_2 = curve_fit(linear, log.(hs), log.(abs.(I2 .- I_exact)), [.5,.5]);


In [ ]:
scatter(log.(hs), log.(abs.(I1 .- I_exact)), label ="First Order", c="blue", legend=:right,
        xlabel = L"\log(h)", ylabel =L"\Delta_k(N)")
plot!(log.(hs), linear((log.(hs)), fit_1.param), label ="", c="blue")


string_1 = latexstring( L"\gamma_1=",@sprintf("%.3f", fit_1.param[2]))


scatter!(log.(hs), log.(abs.(I2 .- I_exact)), label ="Second Order", c="red")
plot!(log.(hs), linear((log.(hs)), fit_2.param), label ="", c="red")

string_2 = latexstring( L"\gamma_2=",@sprintf("%.3f", fit_2.param[2]))
annotate!([(-7, -10, string_1), (-5, -25, string_2)])




In [ ]:
The exponents agree with the expected values.

## Part b

In [ ]:

ϵ = 0
α = .5

hs = []
I1 = []
I2 = []
for n=0:20
    #println(n)
    intsing1 = IntSing(ϵ, α, n, 1)
    push!(I1, intsing1.ifxn(intsing1))
    
    intsing2 = IntSing(ϵ, α, n, 2)
    push!(I2, intsing2.ifxn(intsing2))
    
    push!(hs, intsing2.h)
end

I_exact = 1/(-α + 1)


fit_1 = curve_fit(linear, log.(hs), log.(abs.(I1 .- I_exact)), [.5,.5])
fit_2 = curve_fit(linear, log.(hs), log.(abs.(I2 .- I_exact)), [.5,.5]);


In [ ]:
scatter(log.(hs), log.(abs.(I1 .- I_exact)), label ="First Order", c="blue", legend=:right,
        xlabel = L"\log(h)", ylabel =L"\Delta_k(N)")
plot!(log.(hs), linear(log.(hs), fit_1.param), label ="", c="blue")


string_1 = latexstring( L"\gamma_1=",@sprintf("%.5f", fit_1.param[2]))


scatter!(log.(hs), log.(abs.(I2 .- I_exact)), label ="Second Order", c="red")
plot!(log.(hs), linear(log.(hs), fit_2.param), label ="", c="red", title = L"\alpha = 0.5")

string_2 = latexstring( L"\gamma_2=",@sprintf("%.5f", fit_2.param[2]))

annotate!([(-10, -3, string_1), (-7, -7, string_2)])




In [ ]:
ϵ = 0
α = .75

hs = []
I1 = []
I2 = []
for n=0:20
    #println(n)
    intsing1 = IntSing(ϵ, α, n, 1)
    push!(I1, intsing1.ifxn(intsing1))
    
    intsing2 = IntSing(ϵ, α, n, 2)
    push!(I2, intsing2.ifxn(intsing2))
    
    push!(hs, intsing2.h)
end

I_exact = 1/(-α + 1)


fit_1 = curve_fit(linear, log.(hs), log.(abs.(I1 .- I_exact)), [.5,.5])
fit_2 = curve_fit(linear, log.(hs), log.(abs.(I2 .- I_exact)), [.5,.5]);


In [ ]:
scatter(log.(hs), log.(abs.(I1 .- I_exact)), label ="First Order", c="blue", legend=:right,
        xlabel = L"\log(h)", ylabel =L"\Delta_k(N)")
plot!(log.(hs), linear(log.(hs), fit_1.param), label ="", c="blue")


string_1 = latexstring( L"\gamma_1=",@sprintf("%.5f", fit_1.param[2]))


scatter!(log.(hs), log.(abs.(I2 .- I_exact)), label ="Second Order", c="red")
plot!(log.(hs), linear(log.(hs), fit_2.param), label ="", c="red", title = L"\alpha = 0.75")

string_2 = latexstring( L"\gamma_2=",@sprintf("%.5f", fit_2.param[2]))

annotate!([(-12, -0.5, string_1), (-7, -2, string_2)])




In both cases, the exponent $\gamma$ is
$$ \gamma = 1-\alpha$$

For the open interval formula, the error is dominated by the singular end point at $x=0$:

$$
\Delta I = |I_k(N) - I| \approx \int_0^h f(x) dx
$$
where the integral on the right-hand side is the *exlcuded* contribution to the integral in our forumals. We can solve this explictly to calculate the $h$ dependance.
$$
\int_0^h f(x) dx = \int_0^h \frac{1}{x^\alpha} dx = \frac{x^{1-\alpha}}{1-\alpha}\Big|_0^h =  \frac{h^{1-\alpha}}{1-\alpha}\propto h^{1-\alpha}
$$

Thus, the dominant contribution to the error from excluding the singular point scales as $h^{1-\alpha}$, confirming the results of the above plots. Note that there is still a $\mathcal{O}\left(h^2\right) or \mathcal{O}\left(h^3\right)$ error contribution, but the $\mathcal{O}\left(h^{1-\alpha}\right)$ contribution dominates.

In [ ]:

α = .5
ϵ = 1e-5
hs = []
I1 = []
I2 = []
for n=0:21
    #println(n)
    intsing1 = IntSing(ϵ, α,n, 1)
    push!(I1, intsing1.ifxn(intsing1))
    
    intsing2 = IntSing(ϵ, α, n, 2)
    push!(I2, intsing2.ifxn(intsing2))
    
    push!(hs, intsing2.h)
end

I_exact =  ((1+ϵ)^(1-α)) / (1-α) - ((ϵ)^(1-α)) / (1-α)


fit_1a = curve_fit(linear, log.(hs)[1:4], log.(abs.(I1 .- I_exact))[1:4], [.5,.5])
fit_2a = curve_fit(linear, log.(hs)[1:4], log.(abs.(I2 .- I_exact))[1:4], [.5,.5]);


fit_1b = curve_fit(linear, log.(hs)[18:end], log.(abs.(I1 .- I_exact))[18:end], [.5,.5])
fit_2b = curve_fit(linear, log.(hs)[18:end], log.(abs.(I2 .- I_exact))[18:end], [.5,.5]);


In [ ]:
scatter(log.(hs), log.(abs.(I1 .- I_exact)), label ="First Order", c="blue", legend=:bottomright,
        xlabel = L"\log(h)", ylabel =L"\Delta_k(N)")
plot!(log.(hs)[1:13], linear(log.(hs)[1:13], fit_1a.param), label ="", c="blue")
plot!(log.(hs)[12:end], linear(log.(hs)[12:end], fit_1b.param), label ="", c="blue")


string_1a = latexstring( L"\gamma_1=",@sprintf("%.5f", fit_1a.param[2]))
string_1b = latexstring( L"\gamma_1=",@sprintf("%.5f", fit_1b.param[2]))


scatter!(log.(hs), log.(abs.(I2 .- I_exact)), label ="Second Order", c="red")
plot!(log.(hs)[1:13], linear(log.(hs)[1:13], fit_2a.param), label ="", c="red")
plot!(log.(hs)[12:end], linear(log.(hs)[12:end], fit_2b.param), label ="", c="red")



string_2a = latexstring( L"\gamma_2=",@sprintf("%.5f", fit_2a.param[2]))
string_2b = latexstring( L"\gamma_2=",@sprintf("%.5f", fit_2b.param[2]))


annotate!([(-6, -8, string_1a), (-6, -10, string_2a),
           (-14, -4, string_1b), (-14, -6, string_2b)])

plot!([log(ϵ), log(ϵ)], [0, -23], linestyle=:dash, label = L"x=\log(\epsilon)")




We do indeed see a "cross-over" region, where the scaling of the error switches from $\mathcal{O}\left(h^{1-\alpha}\right)$ to $\mathcal{O}\left(h^2\right) or \mathcal{O}\left(h^3\right)$. 

When $h$ is larger than $\epsilon$ (to the right of the dotted black line), we see the $\mathcal{O}\left(h^{1-\alpha}\right)$ scaling. In this regime, the step size is larger than the deviation from the singular form, so it's as if we don't have the resolution to determine that $f(x)$ is not divergent at $x=0$. 

When $h$ is less than $\epsilon$ (to the left of the dotted black line), we see the $\mathcal{O}\left(h^2\right) and \mathcal{O}\left(h^3\right)$ scaling. In this regime, the step size is small than the deviation from the singular form, so we can resolve the non-divergence of $f(x)$ is at $x=0$. 

# 1) Monte Carlo Integration


In [2]:
using CSV, DataFrames

## Simulation Functions

In [2]:
"""
Sphere structure containing
r_1: radius of sphere
r_2: radius of cylinder
rho_1: density of sphere
rho_2: density of cylinder
z_cyl: positive z position of top of cylinder (h/2)
L: length of box enclosing sphere (2* r_1)
bin_outfile: path for bin average results
res_outfile: path for final average and error bar results
"""
struct Sphere
    r_1::Float64
    r_2::Float64

    rho_1::Float64
    rho_2::Float64
    
    z_cyl::Float64
    
    L::Float64

    bin_outfile::String
    res_outfile::String
end


function Sphere(r_1, r_2, rho_1, rho_2, bin_outfile, res_outfile)
    
    z_cyl = sqrt(r_1^2 - r_2^2) #z position of clyinder found using Pythagorean theorem
    
    sphere = Sphere(r_1, r_2, rho_1, rho_2, z_cyl, 2*r_1,  bin_outfile, res_outfile)
    
end

#Function that returns whether or not point (x, y, z) is in the cylinder
function in_cylinder(sphere, x, y, z)
    ρ = (x^2 + y^2) ^ (1/2) #radius in cylindrical coordiantes
    
    if ρ <= sphere.r_2 && abs(z) <= sphere.z_cyl #condition that point (x, y, z) is in the cylinder
        return true
    else
        return false
    end
end

#value of moment of inertia integrand at point x, y, z, for moment of interatia along given axis
function integrand(sphere, x, y, z, axis)
    
    #perpendicular distance from axis
    if axis == "x"
        r_perp2 = y^2 + z^2
    elseif axis == "z"
        r_perp2 = x^2 + y^2
    else
        throw("Invalid axis of rotation")
    end
    
    
    if in_cylinder(sphere, x, y, z)
        return r_perp2 * sphere.rho_2
    else
        return r_perp2 * sphere.rho_1
    end
end

#Integrate over sphere object using npt points per simulation for nbin simulations 
function integrate(sphere, npt, nbin)
    Ix = []
    Iz = []
    
    
    #open empty bin average output file
    #overwrite if file already exists
    open(sphere.bin_outfile, "w") do f

        for nb = 1:nbin
            
            #generate all npt (x, y, z) samples for the simulation
            #(x, y, z) = 3 random numbers between -L and L
            samples = rand(npt, 3) .* (2*sphere.r_1) .- (sphere.r_1)
            
            #print status of simulation
            if nb % 1000 == 0
                println("Bin number: ",nb)
            end
            
            #initialize monent of intertias for each simulations
            Ix_bin = 0 
            Iz_bin = 0 
            for np = 1:npt
                #select np'th set of (x, y, z) values
                x, y, z = samples[np, :]

                # if (x, y, z) is not in sphere, return zero for value of integrand
                if (x^2 + y^2 + z^2) > sphere.r_1^2
                     Ix_bin += 0
                     Iz_bin += 0
                #else add 1/npt * value of integrand at (x, y, z), for each monent of inertia
                #(dividing by 1/npt in this step takes the avearge over all samples)
                else
                    Ix_bin += integrand(sphere, x, y, z, "x") / npt
                    Iz_bin += integrand(sphere, x, y, z, "z") / npt
                end
            end
            
            #write simulation number, I_z, and I_x to file
            #multiply integrand average by box volume sphere.L^3 to get moment of inertia
            println(f, nb, " ", Iz_bin*(sphere.L^3), " ", Ix_bin*(sphere.L^3))

        end
    end
    
end

function V_sphere(sphere, npt, nbin)
    V = []
    
    for nb in 1:nbin
        V_bin = 0 
        for np in 1:npt
            x, y, z = rand(3) .* (sphere.L) .- (sphere.L/2)
            
            if (x^2 + y^2 + z^2) > sphere.r_1
                 V_bin += 0
            else
                V_bin += 1/npt
            end
        end
        push!(V, V_bin)
    end
    return V
end

function analyze_data(sphere)
    #load bin averages as DataFrame
    df = CSV.read(sphere.bin_outfile, delim = " ", header=false, DataFrame)
    
    
    nbins = [50, 500, 5000]
    
    #calclate statistics for each bin number
    #write statistics to result outfile
    open(sphere.res_outfile, "w") do f
        for nbin in nbins
            Iz_sample = df[1:nbin, "Column2"]
            Ix_sample = df[1:nbin, "Column3"]

            σ_z = sqrt(1/(nbin*(nbin-1)) * sum(Iz_sample .^2 .- mean(Iz_sample)^2))
            σ_x = sqrt(1/(nbin*(nbin-1)) * sum(Ix_sample .^2 .- mean(Ix_sample)^2))
            
            println(f, nbin, " ", mean(Iz_sample), " ", σ_z, " ", mean(Ix_sample), " ", σ_x) 

        end
    end
    println("Statistics exported")
end

analyze_data (generic function with 1 method)

## Initializing Sphere Object

In [3]:
s = Sphere(.05, .01, 8930, 19320, "bin.dat", "res.dat");

## Running 5,000 simulations

In [32]:
@time integrate(s, 10^6,  5000)

Bin number: 1000
Bin number: 2000
Bin number: 3000
Bin number: 4000
Bin number: 5000
334.361077 seconds (5.00 G allocations: 745.070 GiB, 6.49% gc time, 0.03% compilation time)


## Calculating Statistics for $n_{bin}$ = 50, 500, 5,000

In [46]:
analyze_data(s)

Statistics exported


## Comparing Results to Exact Value of $I_z, I_x$

In [47]:
df = CSV.read("res.dat", delim = " ", DataFrame, header=false)

,Column1,Column2,Column3,Column4,Column5
,Int64,Float64,Float64,Float64,Float64
1,50,0.00469272,8.14559e-7,0.00493933,1.04907e-6
2,500,0.00469192,2.61004e-7,0.0049396,3.18356e-7
3,5000,0.00469178,8.74882e-8,0.00493965,9.62373e-8


In [50]:
#Exact I_z
I_cyl_rho2z = 19320 * pi *.01^4 * sqrt(.05^2 - .01^2)
I_cyl_rho1z = 8930 * pi *.01^4 * sqrt(.05^2 - .01^2)

I_sph_rho1 = 8930 * (8 *pi /15) *.05^5 
    
error_z = abs((I_sph_rho1 - I_cyl_rho1z + I_cyl_rho2z) - df[3, "Column2"])

println("Exact Error: ",error_z)
println("Monte Carlo σ_z: ",df[3, "Column3"])

Exact Error: 4.913184445138469e-8
Monte Carlo σ_z: 8.74882243583041e-8


In [51]:
#Exact I_x
I_cyl_rho2x = 19320 * (pi/6) *.01^2 * sqrt(.05^2 - .01^2) * ((4*.05^2) - .01^2)
I_cyl_rho1x = 8930 * (pi/6) *.01^2 * sqrt(.05^2 - .01^2) * ((4*.05^2) - .01^2)


error_x = (I_sph_rho1 - I_cyl_rho1x + I_cyl_rho2x) - df[3, "Column4"]

println("Exact Error: ",error_x)
println("Monte Carlo σ_x: ",df[3, "Column5"])

Exact Error: -5.983015735535474e-8
Monte Carlo σ_x: 9.623728119689116e-8
